# Python DataBase SQLAlchemy


In [8]:
from sqlalchemy import create_engine, text
import pandas as pd

# path to file
DB_FILE = "sakila_master.db"   

# creer d'engine
engine = create_engine(f"sqlite:///{DB_FILE}", echo=False)
print(f"Connexion établie à la base de données : {DB_FILE}\n")

def run_query_df(requete: str) -> pd.DataFrame:
    """
    Exécute une requête SQL et renvoie le résultat sous forme de DataFrame.

    Args:
        requete (str): La requête SQL à exécuter.

    Returns:
        pd.DataFrame: Le DataFrame contenant les données résultantes.

    Raises:
        Exception: Une erreur est affichée si la récupération ou l'exécution
            de la requête échoue.
    """

    try:
        with engine.connect() as conn:
            df = pd.read_sql(text(requete), conn)

            print("--- Résultat de la requête ---")
            print(df.to_markdown(index=False))
            print("-"*70)
        return df

    except Exception as e:
        print(f"Une erreur s'est produite dans la récupération des données...")


Connexion établie à la base de données : sakila_master.db



# Exercices de SQL Pur
---


In [9]:
req = """
SELECT * 
FROM sqlite_master 
WHERE type='table';
"""

df = run_query_df(req)

--- Résultat de la requête ---
| type   | name          | tbl_name      |   rootpage | sql                                                                                                                                      |
|:-------|:--------------|:--------------|-----------:|:-----------------------------------------------------------------------------------------------------------------------------------------|
| table  | actor         | actor         |          2 | CREATE TABLE actor (                                                                                                                     |
|        |               |               |            |   actor_id numeric NOT NULL ,                                                                                                            |
|        |               |               |            |   first_name VARCHAR(45) NOT NULL,                                                                                                   

### Exercices de Lecture (SELECT)

1.  **Affichage, Alias, Ordre et Limite :**
   
      Affichez les **5 titres de films** (`title` de la table `film`) classés par **ordre alphabétique croissant**, en utilisant un **alias** pour la colonne.

In [10]:
req = """
SELECT title AS t
FROM film
ORDER BY title ASC
LIMIT 5;
"""

df = run_query_df(req)


--- Résultat de la requête ---
| t                |
|:-----------------|
| ACADEMY DINOSAUR |
| ACE GOLDFINGER   |
| ADAPTATION HOLES |
| AFFAIR PREJUDICE |
| AFRICAN EGG      |
----------------------------------------------------------------------


2.  **Agrégation**

Calculez 
   - la **moyenne**, 
   - le **minimum**, 
   - le **maximum**, 
   - le **nombre total** et 
   - la **somme** du champ `length` (durée en minutes) de tous les films dans la table `film`.

In [11]:
req = """
SELECT  AVG(length) as mean, 
MIN(length) as min,
MAX(length) as max,
COUNT(*) as count,
SUM(length) as somme
FROM film;
"""
df = run_query_df(req)

--- Résultat de la requête ---
|    mean |   min |   max |   count |   somme |
|--------:|------:|------:|--------:|--------:|
| 115.272 |    46 |   185 |    1000 |  115272 |
----------------------------------------------------------------------


### Exercices de Modification (INSERT / UPDATE)

3.  **Insertion (`INSERT`) avec gestion d'ID et de Date :**
    * **Insérez** un nouvel acteur avec le prénom `"JEAN-CLAUDE"`, le nom `"VANDAMME"`, le nouvel ID calculé, et la date/heure actuelle pour le champ **`last_update`**.
      * **Trouvez** l'**`actor_id` maximum** existant dans la table `actor`, puis **calculez le nouvel ID** en ajoutant 1.
      * **Format de date requis pour le `last_update` :** Utilisez le module `datetime` pour générer la chaîne `YYYY-MM-DD HH:MM:SS`.

In [21]:
# Fonctionnalité supplémentaire
def blob_to_int(x):
    """
    Convertit un objet BLOB (bytes ou bytearray) en entier.

    Args:
        x (bytes | bytearray | any): Valeur à convertir.

    Returns:
        int | any: L'entier obtenu à partir du BLOB, ou la valeur d'origine
        si aucune conversion n'est effectuée.
    """
        
    if isinstance(x, (bytes, bytearray)):
        return int.from_bytes(x, byteorder='little')
    return x

In [24]:
# Trouvez l'actor_id maximum
req = """
SELECT MAX(actor_id) as max
FROM actor;
"""

df = run_query_df(req)


--- Résultat de la requête ---
|   max |
|------:|
|   201 |
----------------------------------------------------------------------


In [25]:
def run_sql(query: str, params: dict = None):
    """
    Exécute une requête SQL (INSERT, UPDATE, DELETE, etc.) avec paramètres.
    
    Args:
        query (str): La requête SQL avec paramètres (:param)
        params (dict, optional): Valeurs à substituer dans la requête
    
    Returns:
        int: Nombre de lignes affectées

    Raises:
        Exception: Affichée si une erreur survient lors de l'exécution ou de
            la validation de la transaction.
    """
    try:
        with engine.begin() as conn:  # begin = transaction auto-commit
            result = conn.execute(text(query), params or {})
            return result.rowcount
    except Exception as e:
        print(f"Erreur SQL: {e}")
        return 0

In [26]:
def insert_into_table(query, data):
    """
    Exécute une requête d'insertion SQL avec les données fournies.

    Args:
        query (str): La requête SQL paramétrée à exécuter.
        data (dict | list | tuple): Les données à insérer. Leur structure doit
            correspondre aux paramètres attendus par la requête.

    Returns:
        None

    Raises:
        Exception: Affichée si une erreur survient lors de l'exécution ou de
            la validation de la transaction.
    """
    try:
        with engine.connect() as conn:
            conn.execute(text(query), data)
  
    except Exception as e:
        print(f"Une erreur s'est produite dans l'insertion' des données...{e}")
    

In [28]:
last_id = blob_to_int(df.iloc[0, 0])

new_id = int(last_id + 1)

data = {"actor_id": new_id,
    "first_name": "JEAN-CLAUDE",
    "last_name": "VANDAMME",
    
}

req = """
INSERT INTO actor (actor_id, first_name, last_name, last_update) 
VALUES (:actor_id, :first_name, :last_name, CURRENT_TIMESTAMP); 
""" 

# insert_into_table(req, data)

4.  **Modification (`UPDATE`) par ID :** **Modifiez** l'acteur que vous venez d'insérer (en le ciblant par son **ID unique**) pour changer son prénom de `"JEAN-CLAUDE"` à `"JC"`.

In [29]:
def simple_update(query, data):
    """
    Выполняет UPDATE-запрос.
    
    :param query: SQL UPDATE с плейсхолдерами (:key)
    :param data: словарь с данными для подстановки
    """
    try:
        with engine.connect() as conn:
            conn.execute(text(query), data)
            conn.commit()
        print("Запись успешно обновлена!")
    except Exception as e:
        print("Ошибка при обновлении:", e)

In [17]:
query_update = """
UPDATE actor
SET first_name = :new_first_name
WHERE actor_id = :actor_id
"""
data_update = {
    "new_first_name": "JC",
     "actor_id": new_id
     }

simple_update(query_update, data_update)


Запись успешно обновлена!


In [30]:
req = """
SELECT * 
FROM actor 
WHERE actor_id > 200;
"""
df = run_query_df(req)


--- Résultat de la requête ---
|   actor_id | first_name   | last_name   | last_update         |
|-----------:|:-------------|:------------|:--------------------|
|        201 | JC           | VANDAMME    | 2025-11-24 15:53:14 |
----------------------------------------------------------------------
